In [1]:
import pandas as pd
import numpy as np

The KBO originated in 1982. I'm going to use pandas read_html() and loop through the season results page for each season to collect the final standings. 

In [2]:
pages = ["6a2bec36", "3ef9029d", "8aa559d8", "8f7d6569", "dffe1d5e", "867da834", "1c9e7c8b", "562cc471", "7f634357",
         "242952f6", "15c2c198", "07c6ddeb", "9798c3f8", "b3541612", "5a6bdf0f", "08d6783b", "8b1281f1", "405d7eba",
         "ec68dffc", "830335e3", "a8cfce5f", "b41c916c", "cdbb609e", "e8747b8f", "3706c744", "94cd40f3", "b9e07969",
         "4e9a4805", "fb11c3eb", "8fb1fec9", "247bbfce", "dbc251cc", "d88d7497", "ac6e830f", "7eb48ea3", "d1e5806d",
         "e942b2f6", "72b8a933"]

column_names = ["Tm", "W", "L", "Ties", "W-L%", "GB", "Year"]

pd.DataFrame(columns = column_names)

year = 2019

seasons = pd.DataFrame(columns = column_names)

for page in pages:

    tables = pd.read_html("https://www.baseball-reference.com/register/league.cgi?id=" + page)

    standings = tables[0]
    standings['Year'] = year
    
    year -= 1
    
    seasons = seasons.append(standings, ignore_index = True)

Since this dataset is small I'm going to write it to a CSV and visually inspect everything in Excel. 

In [3]:
seasons.to_csv('KBO_raw.csv')

The first thing I noticed is that there is an unexpected column "Dream Division." This is showing up because in 1999 and 2000 the league tried a two division format and had a "Magic Division" and "Dream Division." The table on baseball-reference.com is formated differently. I'll fix that, convert the numbers of wins from a string to an intenger, and then calculate cumulative wins for each team. 

In [4]:
seasons.loc[seasons.Year.isin([1999, 2000]), 'Tm'] = seasons.loc[seasons.Year.isin([1999, 2000]), 'Dream Division']
seasons = seasons[seasons.Tm != 'Magic Division']
seasons.drop(columns = ['Dream Division'], inplace = True)
seasons.W = seasons.W.astype('int')
seasons['Total Wins'] = seasons.sort_values('Year').groupby('Tm').cumsum()

The next issue to address is that the franchises switch names all the time which is making our cumulative win totals incorrect.  The Doosan Bears were previously OB Bears, so I'll add the total number of wins the OB Bears had to every value for the Doosan Bears.

In [5]:
seasons.loc[seasons.Tm == 'Doosan Bears', 'Total Wins'] = seasons.loc[seasons.Tm == 'Doosan Bears', 'Total Wins']\
                                                         + seasons.loc[seasons.Tm == 'OB Bears', 'Total Wins']\
                                                           .max()

The Hanwha Eagles used to be the Binggre Eagles and based on everything I find online the spelling should be Binggrae. 

In [6]:
seasons.loc[seasons.Tm == 'Hanwha Eagles', 'Total Wins'] = seasons.loc[seasons.Tm == 'Hanwha Eagles', 'Total Wins']\
                                                          + seasons.loc[seasons.Tm == 'Binggre Eagles', 'Total Wins']\
                                                            .max()

seasons.loc[seasons.Tm == 'Binggre Eagles', 'Tm'] = "Binggrae Eagles"

The Kia Tigers were formerly the Haitai Tigers. At this point I realized it would have been more efficient to split the sponsor and team name into two separate columns. Calculating the cumulative win total on that team name column would have yielded better results. 

In [7]:
seasons.loc[seasons.Tm == 'Kia Tigers', 'Total Wins'] = seasons.loc[seasons.Tm == 'Kia Tigers', 'Total Wins']\
                                                        + seasons.loc[seasons.Tm == 'Haitai Tigers', 'Total Wins']\
                                                          .max()

The Kiwoom Heroes were previously the Nexen Heroes and prior to that they were the Woori Heroes. 

In [8]:
seasons.loc[seasons.Tm == 'Nexen Heroes', 'Total Wins'] = seasons.loc[seasons.Tm == 'Nexen Heroes', 'Total Wins']\
                                                          + seasons.loc[seasons.Tm == 'Woori Heroes', 'Total Wins']\
                                                            .max()

seasons.loc[seasons.Tm == 'Kiwoom Heroes', 'Total Wins'] = seasons.loc[seasons.Tm == 'Kiwoom Heroes', 'Total Wins']\
                                                           + seasons.loc[seasons.Tm == 'Nexen Heroes', 'Total Wins']\
                                                             .max()

The Sammi Superstars became the Chungbo Pintos who became the Pacific Dolphins who became the Hyundai Unicorns. 

In [9]:
seasons.loc[seasons.Tm == 'Chungbo Pintos', 'Total Wins'] = seasons.loc[seasons.Tm == 'Chungbo Pintos', 'Total Wins']\
                                                            + seasons.loc[seasons.Tm == 'Sammi Superstars', 'Total Wins']\
                                                              .max()

seasons.loc[seasons.Tm == 'Pacific Dolphins', 'Total Wins'] = seasons.loc[seasons.Tm == 'Pacific Dolphins', 'Total Wins']\
                                                              + seasons.loc[seasons.Tm == 'Chungbo Pintos', 'Total Wins']\
                                                                .max()

seasons.loc[seasons.Tm == 'Hyundai Unicorns', 'Total Wins'] = seasons.loc[seasons.Tm == 'Hyundai Unicorns', 'Total Wins']\
                                                              + seasons.loc[seasons.Tm == 'Pacific Dolphins', 'Total Wins']\
                                                                .max()

The LG Twins were formerly the MBC Blue dragons. 

In [10]:
seasons.loc[seasons.Tm == 'LG Twins', 'Total Wins'] = seasons.loc[seasons.Tm == 'LG Twins', 'Total Wins']\
                                                      + seasons.loc[seasons.Tm == 'MBC Blue Dragons', 'Total Wins']\
                                                        .max()

I'll write this to a csv and use Power BI for the remaining portion of the project. 

In [11]:
seasons.to_csv('KBO_cleaned.csv')